In [36]:
#%%
import sys
# sys.path.append("/home/baokhanh/")
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

In [117]:

drive_client = GoogleDriveClient( )
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient( )
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

avay_bq_acc = AvayBQServiceAcc()

sa_ana = Path('/home/linhnguyen/.config/ts-dp-v2.json')
adc = Path('/home/linhnguyen/.config/gcloud/application_default_credentials.json')
bq_client = BigqueryClient(billing_project_id='prj-ts-p-analytic-8057',  adc_file_path= adc)
bq_repo = BigqueryService(bq_client)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

channel_id = "C04MYU3L7LN"
user_id = "U0480LX5468"
linh_id = "U047QMYB9TQ"


In [38]:
def dayrange_recon(day_from_curr: int = 25) -> tuple:
    curr_time = datetime.now()
    recon_time =(curr_time - timedelta(day_from_curr))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    return recon_month, start_date, end_date

def minfee_info():
    minfee_month = parse(str(datetime.today() - relativedelta(days=120))).strftime('%Y-%m')
    minfee_start_month = (datetime.today() - relativedelta(days=120)).replace(day=1)
    minfee_end_month = (minfee_start_month + relativedelta(months=1, days=-1)).replace(day=1)
    
    return minfee_month, minfee_start_month, minfee_end_month

In [39]:
recon_month, start_date, end_date = dayrange_recon()
minfee_month, minfee_start_month, minfee_end_month = minfee_info()
print(f'{start_date} - {end_date}')

2023-08-01 - 2023-08-31


In [40]:
# create directory
path = '/home/linhnguyen/04.Reconcile/06.TPB/sms_activate_card/'

if not os.path.exists(f'{path}{recon_month}'):
    os.makedirs(f'{path}{recon_month}')

### Query card list

In [166]:
tpb_query = f"""
with tbl1 as (
  SELECT distinct 
  application_id,
  CAST(JSON_VALUE(result_data, '$.id') AS INTEGER) AS offer_qualifications_id
  FROM `prj-ts-p-lakehouse-8435.dop.application_services`
)
, tbl2 AS (
  SELECT DISTINCT 
  id as offer_qualifications_id
  , CAST(JSON_VALUE(result, '$.other.lead_id') AS INTEGER) AS telco_lead_id
  FROM `prj-ts-p-lakehouse-8435.v1_proxy.offer_qualifications` 
)

, mapping AS (
  SELECT 
  tbl1.application_id
  , tbl2.telco_lead_id
  FROM tbl1 JOIN tbl2 USING(offer_qualifications_id)
)

  select smv.* , mapping.telco_lead_id
  FROM `shared_dop.tpbank_daily_data_approved_and_active_for_recon` smv
  LEFT JOIN mapping USING (application_id)
  WHERE is_actived=1
  AND card_active_date >= date_trunc(date_sub(current_date, interval 4 month), month)  
  AND card_active_date < date_trunc(current_date, month) 

  UNION DISTINCT
  select smv.* , mapping.telco_lead_id
  FROM `shared_dop.tpbank_daily_data_approved_and_active_for_recon` smv
  LEFT JOIN mapping USING (application_id)
  WHERE true
  AND uw_approved_date >= date_trunc(date_sub(current_date, interval 4 month), month)  
  AND uw_approved_date < date_trunc(date_sub(current_date, interval 3 month), month);"""

tpb_df = bq_repo.get_data_from_query_into_pandas(query_str= tpb_query)
tpb_df

,lead_unique_token,lead_phone_number,lead_created_at,product_code,lead_id,application_id,lead_telco_code,lead_source,mkt_channel,utm_campaign,smv_card_category,sms_offer_check,channel,uw_approved_date,card_issued_time,card_active_date,is_issued,is_actived,card_status,telco_lead_id
0,cRFeuRAo,ATj9Eoc/v2TWL/3FnIU8zEEix4vV88VFccBH+DU=,2023-05-04 03:55:47+00:00,tpb_01,0,5710341,vinaphone,organic,EVO,None,VISA CREDIT GOLD EVO CARD (9024),None,EVO,2023-05-05,2023-05-05 17:22:01,2023-05-10,1,1,ACTIVATE,<NA>
1,cRFevv59,AWwJpHUTMz71O2NFUHSpQ01EjefJpcUjaaoQEP8=,2023-05-11 17:33:54+00:00,tpb_01,0,5758927,viettel,organic,FPT,offline_fpt_campaign,VISA CREDIT GOLD EVO CARD (9024),None,OTHER,2023-05-11,2023-05-11 19:18:23,NaT,1,0,None,<NA>
2,kRXr7LmP,AUM3EgBRqV81uQ/hnaZv9BuUV8JXNb6ofAG7R6U=,2023-05-11 14:09:26+00:00,tpb_01,0,5757299,mobifone,organic,SMOLLAN,dop_ts,VISA CREDIT GOLD EVO CARD (9024),None,OTHER,2023-05-11,2023-05-11 16:20:26,2023-05-11,1,1,ACTIVATE,<NA>
3,YRFewxrY,ARJCa94wAfdJHGqtxuJNq5zCpqAIE5MKu5gYKYw=,2023-05-17 09:41:26+00:00,tpb_01,0,5792603,viettel,organic,COCCOC_ADS,search,VISA CREDIT GOLD EVO CARD (9024),None,PERFORMANCE MARKETING,2023-05-17,2023-05-17 10:49:49,2023-05-24,1,1,ACTIVATE,<NA>
4,kRXtj9Gv,AbG41TUpGso0q1JLIsry8YTiblzxOoSBMskXcFE=,2023-06-14 12:18:45+00:00,tpb_01,0,5971875,mobifone,organic,OFFLINE OFFICE BUILDINGS,offline_office_buildings_hado_airport,VISA CREDIT GOLD EVO CARD (9024),None,OTHER,2023-06-14,2023-06-14 12:38:19,2023-06-14,1,1,ACTIVATE,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29541,aRFe7ACw,AeJaKtLsG2fuOdie0FMWqM24PwJlpryVcoO6ZL8=,2023-07-07 14:24:43+00:00,tpb_01,0,6127638,mobifone,esms,SMS-OTHER WITHOUT OFFER,None,VISA CREDIT GOLD EVO CARD (9024),None,SMS WITHOUT OFFER,2023-07-07,2023-07-07 15:32:10,2023-07-17,1,1,ACTIVATE,<NA>
29542,kRXu1cIG,AVu+YUhCayA7toyzdjceIMtPlMoLqjPepn2Aqvo=,2023-07-09 09:12:34+00:00,tpb_01,0,6138756,vinaphone,esms,SMS-OTHER WITHOUT OFFER,None,VISA CREDIT GOLD EVO CARD (9024),None,SMS WITHOUT OFFER,2023-07-09,2023-07-09 10:23:48,2023-07-12,1,1,ACTIVATE,<NA>
29543,YRFfH-q6,AXL6e+pCd78LOridQqI8OWCnsamaMVRWHR5/fJo=,2023-08-31 10:11:07+00:00,tpb_01,0,6552919,viettel,esms,SMS-OTHER WITHOUT OFFER,zalo,VISA EVO SILVER CARD (9023),None,SMS WITHOUT OFFER,2023-08-31,2023-08-31 11:42:33,2023-08-31,1,1,ACTIVATE,<NA>
29544,QiK91rBS,AeIsV4Klsle0KtB94hiS4e+mP99+a+wgkNBO6hQ=,2023-07-06 21:24:18+00:00,tpb_01,0,6122245,mobifone,esms,SMS-OTHER WITHOUT OFFER,None,VISA CREDIT GOLD EVO CARD (9024),None,SMS WITHOUT OFFER,2023-07-07,2023-07-07 08:48:08,2023-07-11,1,1,ACTIVATE,<NA>


In [170]:
tpb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29546 entries, 0 to 29545
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   lead_unique_token  29546 non-null  object             
 1   lead_phone_number  29546 non-null  object             
 2   lead_created_at    29546 non-null  datetime64[ns, UTC]
 3   product_code       29546 non-null  object             
 4   lead_id            29546 non-null  Int64              
 5   application_id     29546 non-null  Int64              
 6   lead_telco_code    29546 non-null  object             
 7   lead_source        29546 non-null  object             
 8   mkt_channel        29546 non-null  object             
 9   utm_campaign       19761 non-null  object             
 10  smv_card_category  29546 non-null  object             
 11  sms_offer_check    7193 non-null   object             
 12  channel            29546 non-null  object     

In [171]:
tpb_sms = tpb_df[(tpb_df['sms_offer_check'] == 'GET_SMS_OFFER') &
                  (tpb_df['card_active_date'] <= end_date) &
                  (tpb_df['card_active_date'] >= start_date)]

tpb_sms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 22122 to 27598
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   lead_unique_token  243 non-null    object             
 1   lead_phone_number  243 non-null    object             
 2   lead_created_at    243 non-null    datetime64[ns, UTC]
 3   product_code       243 non-null    object             
 4   lead_id            243 non-null    Int64              
 5   application_id     243 non-null    Int64              
 6   lead_telco_code    243 non-null    object             
 7   lead_source        243 non-null    object             
 8   mkt_channel        243 non-null    object             
 9   utm_campaign       2 non-null      object             
 10  smv_card_category  243 non-null    object             
 11  sms_offer_check    243 non-null    object             
 12  channel            243 non-null    object   

In [172]:
if tpb_sms[tpb_sms['telco_lead_id'].isna()].empty:
    print('There is NO N/A telco_lead_id')
else:
    num_non_na_rows = len(tpb_sms) - len(tpb_sms[tpb_sms['telco_lead_id'].isna()])
    print(f'There is {num_non_na_rows} SMS W OFFER leads does NOT has telco_lead_id')


There is NO N/A telco_lead_id


In [173]:
if tpb_sms[tpb_sms['lead_phone_number'].duplicated()].empty:
    print('There is NO duplicated lead_phone_number')
else:
    num_dup_rows = len(tpb_sms) - len(tpb_sms[tpb_sms['lead_phone_number'].duplicated()])
    print(f'There is {num_dup_rows} duplicated lead_phone_number')

There is NO duplicated lead_phone_number


### Add telco_lead_id & ts_lead_id from phone_infos

In [174]:
v1_query = f"""
select distinct
CAST(id as INT64) as lead_id
, phone_number
, CAST(telco_lead_id as INT64) as telco_lead_id
, date(created_at) as lead_created_at
from `prj-ts-p-lakehouse-8435.v1_proxy.phone_infos`
where true
and bank_code = 'tpbank'
                        ;
"""
v1_df = bq_repo.get_data_from_query_into_pandas(query_str = v1_query)
v1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351804 entries, 0 to 351803
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   lead_id          351804 non-null  Int64 
 1   phone_number     351804 non-null  object
 2   telco_lead_id    351804 non-null  Int64 
 3   lead_created_at  351804 non-null  dbdate
dtypes: Int64(2), dbdate(1), object(1)
memory usage: 11.4+ MB


In [175]:
v1_df

,lead_id,phone_number,telco_lead_id,lead_created_at
0,14105430,AVCJnagprOHjMGN45t2ZUD67Zbm4Yf3r3kHEeEY=,1416688,2023-05-25
1,14114619,AU3Dyl880M5WUwlGrkngUOXOcYuNmNi2fFG4sU4=,1422770,2023-05-28
2,14032516,AdPABguZMlofXIdwrLrrRo9gfci66VHU1DVDUVA=,3402779,2023-05-04
3,12147213,ASI6AZGN3uFDxJ7XbZz2tZ/TY5ZX7He9sk5/0aY=,269362,2022-06-29
4,12927082,AbwqPK/OyxO7GxWc+UDQYki10u/+e2m9/rKANLk=,636927,2022-10-02
...,...,...,...,...
351799,13816788,AZcICwCvUHwklUcZpgCu7wzM3LnSXp9NeSbs+uc=,3335120,2023-03-15
351800,13814512,AT+kJJRIvQjD3bKWx+rebRXoIqTslkOfA1DCMto=,1271004,2023-03-15
351801,13812637,AWuK+3tMoBMLEpN64Yuswr310w3kj3tQxpHka5A=,21391094,2023-03-15
351802,13810433,AXbUZqNzt+yPqfkD57caHCiw0JSjr9SX8HE/GKk=,21385970,2023-03-15


In [176]:
def generate_tpb_sms_map(tpb_sms, v1_df):
    def to_categorical(tpb_sms_map):
        return pd.merge(tpb_sms_map.drop(columns=['lead_id']), 
                         v1_df[['phone_number', 'telco_lead_id', 'lead_id']],
                         how='left',
                         left_on=['telco_lead_id', 'lead_phone_number'],
                         right_on=['telco_lead_id', 'phone_number']).dropna(subset=['telco_lead_id']).drop_duplicates()

    tpb_sms_map = to_categorical(tpb_sms)

    def to_categorical_na(tpb_sms):
        tpb_sms_na = tpb_sms[tpb_sms['telco_lead_id'].isna()].drop(columns=['lead_id', 'telco_lead_id'])
        return pd.merge(tpb_sms_na, 
                         v1_df[['phone_number', 'telco_lead_id', 'lead_id']],
                         how='left',
                         left_on='lead_phone_number',
                         right_on='phone_number').drop_duplicates()

    tpb_sms_map_na = to_categorical_na(tpb_sms)

    def combine_tpb_sms_maps(tpb_sms_map, tpb_sms_map_na):
        tpb_sms_map_full = pd.concat([tpb_sms_map, tpb_sms_map_na])
        return tpb_sms_map_full.drop_duplicates()

    tpb_sms_map_full = combine_tpb_sms_maps(tpb_sms_map, tpb_sms_map_na)

    return tpb_sms_map_full

tpb_sms_map_full = generate_tpb_sms_map(tpb_sms, v1_df)

display(tpb_sms_map_full.head())

,lead_unique_token,lead_phone_number,lead_created_at,product_code,application_id,lead_telco_code,lead_source,mkt_channel,utm_campaign,smv_card_category,sms_offer_check,channel,uw_approved_date,card_issued_time,card_active_date,is_issued,is_actived,card_status,telco_lead_id,phone_number,lead_id
0,eRFevuHs,AeoDfTMVXBAu/rVv8e+dXDQZnvk5X+Kjkw8sovI=,2023-05-11 15:41:27+00:00,tpb_01,5758013,viettel,esms,SMS-VT WITH OFFER,None,VISA EVO SILVER CARD (9023),GET_SMS_OFFER,SMS WITH OFFER,2023-05-11,2023-05-11 19:55:07,2023-08-26,1,1,ACTIVATE,21945219,AeoDfTMVXBAu/rVv8e+dXDQZnvk5X+Kjkw8sovI=,14059860
1,cRFbazKN,Ae4RTMn0zffl+FQg9OAt7OZ8VGMz/tEHKXapTAQ=,2022-09-15 08:33:23+00:00,tpb_01,3584808,viettel,esms,SMS-VT WITH OFFER,None,VISA EVO SILVER CARD (9023),GET_SMS_OFFER,SMS WITH OFFER,2022-09-15,2022-09-15 09:44:49,2023-08-10,1,1,ACTIVATE,19314131,Ae4RTMn0zffl+FQg9OAt7OZ8VGMz/tEHKXapTAQ=,12775081
2,aRFerWFO,ASLo8tLZWCVXYPsD8nn//AsSxtFVy+Gp2x+YHAQ=,2023-04-18 15:24:15+00:00,tpb_01,5614633,viettel,esms,SMS-VT WITH OFFER,None,VISA EVO SILVER CARD (9023),GET_SMS_OFFER,SMS WITH OFFER,2023-04-18,2023-04-18 17:37:30,2023-08-09,1,1,ACTIVATE,21745418,ASLo8tLZWCVXYPsD8nn//AsSxtFVy+Gp2x+YHAQ=,13972707
3,aRFe5dJS,AVKmf9ABk7oVXkHw6M5sXef3oRA2aLB6KgXzUBM=,2023-06-28 10:29:26+00:00,tpb_01,6077002,viettel,esms,SMS-VT WITH OFFER,None,VISA CREDIT GOLD EVO CARD (9024),GET_SMS_OFFER,SMS WITH OFFER,2023-06-28,2023-06-28 11:42:48,2023-08-11,1,1,ACTIVATE,22262115,AVKmf9ABk7oVXkHw6M5sXef3oRA2aLB6KgXzUBM=,14207455
4,UiK9rBZY,AWxd+d9fn83Hzvp46Xyi6NIUo/nmjz03bA4Stkw=,2023-06-10 16:10:58+00:00,tpb_01,5947749,viettel,esms,SMS-VT WITH OFFER,None,VISA EVO SILVER CARD (9023),GET_SMS_OFFER,SMS WITH OFFER,2023-06-12,2023-06-12 11:04:15,2023-08-18,1,1,ACTIVATE,22149555,AWxd+d9fn83Hzvp46Xyi6NIUo/nmjz03bA4Stkw=,14152439


### ADD PRICING

In [177]:
def transform_tpb_sms_map(tpb_sms_map_full):
    def format_dates(dt_col, fmt):
        return pd.to_datetime(dt_col, format='%Y-%m-%d').dt.strftime(fmt)
    
    def determine_fee(row):
        if row['is_actived'] == 1 and row['lead_telco_code'] == 'mobifone':
            return 300000
        elif row['is_actived'] == 1 and row['lead_telco_code'] in ['viettel', 'vinaphone']:
            return 450000
        else:
            return 0

    tpb_sms_map_full['cycle_active'] = (
        pd.to_datetime(tpb_sms_map_full['card_active_date']).dt.strftime('01.%m.%Y')
        + ' - ' +
        pd.to_datetime(tpb_sms_map_full['card_active_date']).dt.to_period('M').dt.to_timestamp('M').dt.strftime('%d.%m.%Y')
    )

    tpb_sms_map_full['lead_created_date'] = pd.to_datetime(tpb_sms_map_full['lead_created_at']).dt.date
    tpb_sms_map_full['card_issued_date'] = pd.to_datetime(tpb_sms_map_full['card_active_date']).dt.date
    tpb_sms_map_full['card_active_date'] = pd.to_datetime(tpb_sms_map_full['card_active_date']).dt.date
    tpb_sms_map_full['fee_no_vat'] = [determine_fee(row) for idx, row in tpb_sms_map_full.iterrows()]
    tpb_sms_map_full['fee_w_vat'] = tpb_sms_map_full.apply(lambda row: row['fee_no_vat'] * 1.1 if row['is_actived'] == 1 else 0, axis=1)

    return tpb_sms_map_full

tpb_sms_map_full = transform_tpb_sms_map(tpb_sms_map_full)

tpb_sms_map_full[['lead_telco_code', 'fee_no_vat', 'fee_w_vat']].drop_duplicates()


,lead_telco_code,fee_no_vat,fee_w_vat
0,viettel,450000,495000.0
60,mobifone,300000,330000.0
190,vinaphone,450000,495000.0


# DETAILED FILES

### 1. Mobifone

In [178]:
mbf_detail = tpb_sms_map_full \
    .query("lead_telco_code == 'mobifone'") \
    .loc[:, ['application_id','telco_lead_id', 'lead_unique_token', 'lead_telco_code', 'lead_created_date', 'card_active_date', 'fee_no_vat', 'fee_w_vat']] \
    .rename(columns={'application_id':'app_id', 'telco_lead_id': 'lead_id', 'lead_unique_token': 'lead_unique_token', 'card_active_date': 'actived_date'}) \
    .sort_values(by='actived_date', ascending=False) \
    .assign(active_status='ACTIVED',
            fee_no_vat=pd.to_numeric(tpb_sms_map_full['fee_no_vat']),
            fee_w_vat=pd.to_numeric(tpb_sms_map_full['fee_w_vat']))


#Export to xlsx and drive
mbf_detail.to_excel(f"{path}{recon_month}/TPB_MBF_card_active_monthly_report_{recon_month}.xlsx", index=False)

mbf_ftp = mbf_detail[['lead_id', 'lead_unique_token', 'lead_telco_code','lead_created_date', 'actived_date', 'fee_no_vat', 'fee_w_vat','active_status']]
mbf_ftp.rename(columns = {'lead_unique_token': 'phone_number'}, inplace = True)
mbf_ftp.to_excel(f"{path}{recon_month}/FTP_TPB_MBF_card_active_monthly_report_{recon_month}.xlsx", index=False)

mbf_detail[['lead_telco_code', 'fee_no_vat', 'fee_w_vat']].drop_duplicates()

/tmp/ipykernel_1453865/3506566113.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbf_ftp.rename(columns = {'lead_unique_token': 'phone_number'}, inplace = True)


,lead_telco_code,fee_no_vat,fee_w_vat
60,mobifone,300000,330000.0


In [179]:
def create_or_get_folder(recon_month, parent_folder_id):
    # Check if folder exists
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    return folder_id[0]

def create_or_get_workbook(parent_folder_id, wb_name):
    wb_check = drive_repo.list_file_names(parent_folder_id= parent_folder_id)
    if wb_name not in wb_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = wb_name,parent_folder_id = parent_folder_id)
    else:
        spreadsheet_key = drive_repo.get_files(file_name = wb_name,parent_folder_id = parent_folder_id)
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key[0])
    return wb

def create_or_update_sheet(wb, new_sheet, input_df):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    if 'Sheet1' in sheet_names: 
        wb.del_worksheet(wb.worksheet('Sheet1')) 
    else:
        pass
    return

In [180]:
mbf_detail

,app_id,lead_id,lead_unique_token,lead_telco_code,lead_created_date,actived_date,fee_no_vat,fee_w_vat,active_status
60,6190925,3510257,aRFe87pt,mobifone,2023-07-17,2023-08-31,300000,330000.0,ACTIVED
138,4819378,3231959,UiK8mJsj,mobifone,2023-01-14,2023-08-31,300000,330000.0,ACTIVED
153,6217455,3516356,YRFe9vd-,mobifone,2023-07-20,2023-08-30,300000,330000.0,ACTIVED
136,4962097,3259991,YRFeXbmJ,mobifone,2023-02-03,2023-08-30,300000,330000.0,ACTIVED
80,4064941,3080459,eRFb4GrR,mobifone,2022-10-31,2023-08-28,300000,330000.0,ACTIVED
...,...,...,...,...,...,...,...,...,...
152,6264015,3519194,eRFe_KZ4,mobifone,2023-07-26,2023-08-01,300000,330000.0,ACTIVED
107,6262971,3519068,aRFe_IXY,mobifone,2023-07-26,2023-08-01,300000,330000.0,ACTIVED
110,6274241,3519767,QiK9-8wf,mobifone,2023-07-27,2023-08-01,300000,330000.0,ACTIVED
117,6037192,3484508,UiK9weyK,mobifone,2023-06-23,2023-08-01,300000,330000.0,ACTIVED


In [181]:
sms_folder_id = '1CHmqLEPlF6WgE6_HQcn7zQ03Qv31Bg-h'
folder_id = create_or_get_folder(recon_month=recon_month, parent_folder_id=sms_folder_id)
TPB_MBF_wb = create_or_get_workbook(parent_folder_id = folder_id, wb_name = "TPB_MBF_card_active_monthly_report_" + recon_month)
create_or_update_sheet(wb = TPB_MBF_wb, new_sheet = "Details", input_df = mbf_detail)



### 2. Viettel

In [182]:
vt_detail = tpb_sms_map_full \
    .query("lead_telco_code == 'viettel'") \
    .loc[:, ['application_id','telco_lead_id', 'lead_unique_token', 'lead_telco_code', 'lead_created_date', 'card_active_date', 'fee_no_vat', 'fee_w_vat']] \
    .rename(columns={'application_id':'app_id', 'telco_lead_id': 'lead_id', 'card_active_date': 'actived_date'}) \
    .sort_values(by='actived_date', ascending=False) \
    .assign(active_status='ACTIVED',
            fee_no_vat=pd.to_numeric(tpb_sms_map_full['fee_no_vat']),
            fee_w_vat=pd.to_numeric(tpb_sms_map_full['fee_w_vat']))


vt_detail.to_excel(f"{path}{recon_month}/TPB_VT_detail_active_leads_{recon_month}.xlsx", index=False)
vt_detail[['lead_telco_code', 'fee_no_vat', 'fee_w_vat']].drop_duplicates()


,lead_telco_code,fee_no_vat,fee_w_vat
17,viettel,450000,495000.0


In [183]:
vt_detail

,app_id,lead_id,lead_unique_token,lead_telco_code,lead_created_date,actived_date,fee_no_vat,fee_w_vat,active_status
17,4916756,20975465,eRFeWDCh,viettel,2023-02-01,2023-08-30,450000,495000.0,ACTIVED
47,5356294,21361613,cRFejdgx,viettel,2023-03-13,2023-08-28,450000,495000.0,ACTIVED
26,4235696,20353727,YRFeBQ2D,viettel,2022-11-17,2023-08-28,450000,495000.0,ACTIVED
44,5498873,21603545,cRFenz_J,viettel,2023-04-03,2023-08-28,450000,495000.0,ACTIVED
15,3086549,18450848,YRFa8Y1T,viettel,2022-07-26,2023-08-27,450000,495000.0,ACTIVED
0,5758013,21945219,eRFevuHs,viettel,2023-05-11,2023-08-26,450000,495000.0,ACTIVED
11,4801949,20866040,UiK8lFne,viettel,2023-01-12,2023-08-25,450000,495000.0,ACTIVED
16,3008411,18326762,cRFa3nm1,viettel,2022-07-17,2023-08-25,450000,495000.0,ACTIVED
7,5444034,21512450,eRFemI4V,viettel,2023-03-25,2023-08-25,450000,495000.0,ACTIVED
31,5422758,21477023,QiK9K-pt,viettel,2023-03-22,2023-08-24,450000,495000.0,ACTIVED


In [184]:
#Export to xlsx and drive
TPB_VT_wb = create_or_get_workbook(parent_folder_id = folder_id, wb_name = "TPB_VT_detail_active_leads_" + recon_month)
create_or_update_sheet(wb = TPB_VT_wb, new_sheet = "Details", input_df = vt_detail)


### 3. VNPT

In [216]:
from connection import TRACK_DB_CONN


# Execute the SQL query to retrieve the data
rq_vnpt = pd.read_sql("select id as telco_lead_id,\
                        created_at,\
                        JSON_VALUE(downstream_param,'$.lead_id') as vnpt_lead_id\
                  from telco_gateway_vinaphone.general_requests\
                  where client_code = 'tpbank' and request_type = 'forward_lead'", con=TRACK_DB_CONN)

# # Print the first few rows of the data
# display(rq_vnpt.head())
rq_vnpt['vnpt_lead_id'] = rq_vnpt['vnpt_lead_id'].astype('int64')
rq_vnpt

,telco_lead_id,created_at,vnpt_lead_id
0,69495,2022-04-20 10:22:06,71117
1,74638,2022-04-22 14:25:30,74504
2,74639,2022-04-22 14:25:30,74501
3,74647,2022-04-22 14:35:38,74513
4,74672,2022-04-22 14:55:53,74525
...,...,...,...
112313,1492398,2023-06-30 21:39:14,642458
112314,1492485,2023-07-03 19:38:13,642461
112315,1492496,2023-07-04 08:17:11,642464
112316,1492558,2023-07-05 06:40:39,642467


In [217]:
rq_vnpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112318 entries, 0 to 112317
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   telco_lead_id  112318 non-null  int64         
 1   created_at     112318 non-null  datetime64[ns]
 2   vnpt_lead_id   112318 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.6 MB


In [218]:
rq_vnpt['vnpt_lead_id'] = rq_vnpt['vnpt_lead_id'].astype('int64')

In [219]:
vnpt_detail = tpb_sms_map_full[tpb_sms_map_full["lead_telco_code"] == "vinaphone"]\
    .merge(rq_vnpt[["telco_lead_id", "vnpt_lead_id"]], left_on="telco_lead_id", right_on="vnpt_lead_id", how="left")\
    .filter(["lead_telco_code","application_id", "lead_unique_token", "vnpt_lead_id", "telco_lead_id", "lead_created_date", "uw_approved_date", "card_issued_date", "card_active_date", "cycle_active", "fee_no_vat", "fee_w_vat"])

display(vnpt_detail.head())


,lead_telco_code,application_id,lead_unique_token,vnpt_lead_id,lead_created_date,uw_approved_date,card_issued_date,card_active_date,cycle_active,fee_no_vat,fee_w_vat
0,vinaphone,6034520,UiK9wUWN,636116,2023-06-23,2023-06-23,2023-08-16,2023-08-16,01.08.2023 - 31.08.2023,450000,495000.0
1,vinaphone,2845461,kRWraxV0,185300,2022-06-29,2022-06-30,2023-08-17,2023-08-17,01.08.2023 - 31.08.2023,450000,495000.0
2,vinaphone,6005231,aRFe3Q95,630578,2023-06-19,2023-06-19,2023-08-13,2023-08-13,01.08.2023 - 31.08.2023,450000,495000.0
3,vinaphone,6030537,QiK9wEyZ,635471,2023-06-22,2023-06-22,2023-08-09,2023-08-09,01.08.2023 - 31.08.2023,450000,495000.0
4,vinaphone,5549816,YRFepXfE,583061,2023-04-10,2023-04-10,2023-08-25,2023-08-25,01.08.2023 - 31.08.2023,450000,495000.0


In [220]:
tpb_sms_map_full[tpb_sms_map_full["lead_telco_code"] == "vinaphone"].count()

lead_unique_token    53
lead_phone_number    53
lead_created_at      53
product_code         53
application_id       53
lead_telco_code      53
lead_source          53
mkt_channel          53
utm_campaign          0
smv_card_category    53
sms_offer_check      53
channel              53
uw_approved_date     53
card_issued_time     53
card_active_date     53
is_issued            53
is_actived           53
card_status          53
telco_lead_id        53
phone_number          0
lead_id               0
cycle_active         53
lead_created_date    53
card_issued_date     53
fee_no_vat           53
fee_w_vat            53
dtype: int64

In [221]:
vnpt_detail.count()

lead_telco_code      53
application_id       53
lead_unique_token    53
vnpt_lead_id         53
lead_created_date    53
uw_approved_date     53
card_issued_date     53
card_active_date     53
cycle_active         53
fee_no_vat           53
fee_w_vat            53
dtype: int64

In [222]:
vnpt_missing = vnpt_detail[vnpt_detail['vnpt_lead_id'].isna()]
vnpt_missing

,lead_telco_code,application_id,lead_unique_token,vnpt_lead_id,lead_created_date,uw_approved_date,card_issued_date,card_active_date,cycle_active,fee_no_vat,fee_w_vat


In [223]:
vnpt_detail = vnpt_detail[vnpt_detail['vnpt_lead_id'].notna()]
vnpt_detail.count()

lead_telco_code      53
application_id       53
lead_unique_token    53
vnpt_lead_id         53
lead_created_date    53
uw_approved_date     53
card_issued_date     53
card_active_date     53
cycle_active         53
fee_no_vat           53
fee_w_vat            53
dtype: int64

In [224]:
vnpt_detail.to_excel(f"{path}{recon_month}/TPB_VNPT_detail_active_leads_{recon_month}.xlsx", index=False)
vnpt_detail.count()

lead_telco_code      53
application_id       53
lead_unique_token    53
vnpt_lead_id         53
lead_created_date    53
uw_approved_date     53
card_issued_date     53
card_active_date     53
cycle_active         53
fee_no_vat           53
fee_w_vat            53
dtype: int64

In [225]:
#Export to xlsx and drive
TPB_VNPT_wb = create_or_get_workbook(parent_folder_id = folder_id, wb_name = "TPB_VNPT_card_active_detail_" + recon_month)
create_or_update_sheet(wb = TPB_VNPT_wb, new_sheet = "Details", input_df = vnpt_detail)

# SUMMARY TELCO LEAD COST

In [226]:
def summarize_vnpt(df, pricing_no_vat, pricing_vat):
    summary_df = df \
        .assign(lead_created_at=pd.to_datetime(df['lead_created_date']).dt.strftime('%Y-%m'))\
        .groupby(['cycle_active', 'lead_created_at']) \
        .agg(total_cards=('cycle_active', 'count'),
             fee_no_VAT=('fee_no_vat', lambda x: pricing_no_vat * x.count()),
             fee_w_VAT=('fee_w_vat', lambda x: pricing_vat * x.count())) \
        .reset_index()

    total_row = summary_df[['total_cards', 'fee_no_VAT', 'fee_w_VAT']].sum()
    total_row['cycle_active'] = 'Total'
    total_row['lead_created_at'] = ''

    summary_df = summary_df.append(total_row, ignore_index=True)

    return summary_df;

summary_df_vnpt = summarize_vnpt(vnpt_detail, pricing_no_vat=450000, pricing_vat=450000*1.1)
summary_df_vnpt


/tmp/ipykernel_1453865/2779113235.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(total_row, ignore_index=True)


,cycle_active,lead_created_at,total_cards,fee_no_VAT,fee_w_VAT
0,01.08.2023 - 31.08.2023,2022-04,1.0,450000.0,495000.0
1,01.08.2023 - 31.08.2023,2022-05,2.0,900000.0,990000.0
2,01.08.2023 - 31.08.2023,2022-06,1.0,450000.0,495000.0
3,01.08.2023 - 31.08.2023,2022-07,1.0,450000.0,495000.0
4,01.08.2023 - 31.08.2023,2022-08,2.0,900000.0,990000.0
5,01.08.2023 - 31.08.2023,2022-09,4.0,1800000.0,1980000.0
6,01.08.2023 - 31.08.2023,2022-11,3.0,1350000.0,1485000.0
7,01.08.2023 - 31.08.2023,2022-12,3.0,1350000.0,1485000.0
8,01.08.2023 - 31.08.2023,2023-01,4.0,1800000.0,1980000.0
9,01.08.2023 - 31.08.2023,2023-02,1.0,450000.0,495000.0


In [227]:
def summarize_df(tpb_sms_map_full, lead_telco_code, pricing_no_vat, pricing_vat):
    summary_df = tpb_sms_map_full \
        .query("lead_telco_code == @lead_telco_code") \
        .assign(lead_created_at=pd.to_datetime(tpb_sms_map_full['lead_created_at']).dt.strftime('%Y-%m')) \
        .groupby(['cycle_active', 'lead_created_at']) \
        .agg(total_cards=('cycle_active', 'count'),
             fee_no_VAT=('fee_no_vat', lambda x: pricing_no_vat * x.count()),
             fee_w_VAT=('fee_w_vat', lambda x: pricing_vat * x.count())) \
        .reset_index()

    total_row = summary_df[['total_cards', 'fee_no_VAT', 'fee_w_VAT']].sum()
    total_row['cycle_active'] = 'Total'
    total_row['lead_created_at'] = ''

    summary_df = summary_df.append(total_row, ignore_index=True)

    return summary_df;

summary_df_mbf = summarize_df(tpb_sms_map_full, lead_telco_code='mobifone', pricing_no_vat=300000, pricing_vat=330000)
summary_df_mbf


/tmp/ipykernel_1453865/1204769981.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(total_row, ignore_index=True)


,cycle_active,lead_created_at,total_cards,fee_no_VAT,fee_w_VAT
0,01.08.2023 - 31.08.2023,2022-07,1,300000,330000
1,01.08.2023 - 31.08.2023,2022-08,1,300000,330000
2,01.08.2023 - 31.08.2023,2022-09,1,300000,330000
3,01.08.2023 - 31.08.2023,2022-10,3,900000,990000
4,01.08.2023 - 31.08.2023,2022-11,1,300000,330000
5,01.08.2023 - 31.08.2023,2022-12,1,300000,330000
6,01.08.2023 - 31.08.2023,2023-01,4,1200000,1320000
7,01.08.2023 - 31.08.2023,2023-02,3,900000,990000
8,01.08.2023 - 31.08.2023,2023-03,4,1200000,1320000
9,01.08.2023 - 31.08.2023,2023-04,2,600000,660000


In [228]:
summary_df_vt = summarize_df(tpb_sms_map_full, lead_telco_code='viettel', pricing_no_vat=450000, pricing_vat=495000)
summary_df_vt

/tmp/ipykernel_1453865/1204769981.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(total_row, ignore_index=True)


,cycle_active,lead_created_at,total_cards,fee_no_VAT,fee_w_VAT
0,01.08.2023 - 31.08.2023,2022-05,4,1800000,1980000
1,01.08.2023 - 31.08.2023,2022-07,3,1350000,1485000
2,01.08.2023 - 31.08.2023,2022-09,6,2700000,2970000
3,01.08.2023 - 31.08.2023,2022-10,2,900000,990000
4,01.08.2023 - 31.08.2023,2022-11,5,2250000,2475000
5,01.08.2023 - 31.08.2023,2022-12,2,900000,990000
6,01.08.2023 - 31.08.2023,2023-01,2,900000,990000
7,01.08.2023 - 31.08.2023,2023-02,4,1800000,1980000
8,01.08.2023 - 31.08.2023,2023-03,8,3600000,3960000
9,01.08.2023 - 31.08.2023,2023-04,6,2700000,2970000


In [229]:
sheet_id = "1bn-3H9ZQ5NMPp56yiYw2nCoJtbFRUzjLq6FcjOddPMY"
wb = ggsheet_repo.open_spreadsheet(sheet_id)
# ws = wb.add_worksheet(title=recon_month, rows=100, cols=50)
ws = wb.worksheet('2023-08')
ws.update('A1', 'VT')
ws.update('H1', 'VNPT')
ws.update('O1', 'MBF')

ggsheet_repo.write_df_to_sheet(ws, summary_df_vt, 'A2')

ggsheet_repo.write_df_to_sheet(ws, summary_df_vnpt, "H2")

ggsheet_repo.write_df_to_sheet(ws, summary_df_mbf, "O2")


# VNPT MINFEE

In [230]:
vnpt_sent_query = f""" 
WITH tbl1 AS (
  SELECT distinct 
  application_id,
  CAST(JSON_VALUE(result_data, '$.id') AS INTEGER) AS offer_qualifications_id
  FROM `prj-ts-p-lakehouse-8435.dop.application_services`
)
, tbl2 AS (
  SELECT DISTINCT 
  id as offer_qualifications_id
  , CAST(JSON_VALUE(result, '$.other.lead_id') AS INTEGER) AS telco_lead_id
  FROM `prj-ts-p-lakehouse-8435.v1_proxy.offer_qualifications` 
)

, mapping AS (
  SELECT 
  tbl1.application_id
  , tbl2.telco_lead_id
  FROM tbl1 JOIN tbl2 USING(offer_qualifications_id)
)

  select smv.* , mapping.telco_lead_id
  FROM `shared_dop.tpbank_daily_data_approved_and_active_for_recon` smv
  LEFT JOIN mapping USING (application_id)
  WHERE true 
  and date(lead_created_at) >= date_trunc(date_sub(current_date, interval 4 month), month) 
  and date(lead_created_at) < date_trunc(date_sub(current_date, interval 3 month), month) 
  and (card_active_date < date_trunc(current_date, month)  OR card_active_date is null)
  and lead_telco_code = 'vinaphone'
  and sms_offer_check = 'GET_SMS_OFFER'
"""

vnpt_sent_list = bq_repo.get_data_from_query_into_pandas(query_str= vnpt_sent_query)

In [231]:
vnpt_sent_list

,lead_unique_token,lead_phone_number,lead_created_at,product_code,lead_id,application_id,lead_telco_code,lead_source,mkt_channel,utm_campaign,smv_card_category,sms_offer_check,channel,uw_approved_date,card_issued_time,card_active_date,is_issued,is_actived,card_status,telco_lead_id
0,YRFezQ1M,AbtUALhVZg7R1i/HqMXINu6Ccthc2Jykngz6mNg=,2023-05-31 09:27:10+00:00,tpb_01,0,5874089,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,614534
1,eRFeyEAU,AXW2hthsShnIqLkaH/+eecdDQbCELGuPtbMCPJc=,2023-05-24 11:43:10+00:00,tpb_01,0,5834754,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610322
2,YRFeyBPM,AQforAYqU8wyPpIctS509TOTz+9Ft9V7ql3ciH8=,2023-05-24 08:56:50+00:00,tpb_01,0,5833337,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610091
3,eRFex1-w,AXl7HrMYMu8ZevRazrDPrYGdhUm8Qpin1JjlqBQ=,2023-05-23 09:57:55+00:00,tpb_01,0,5827574,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,609569
4,QiK9jTM6,ARNvC20nyCzrIvARB1JRGbF/SQ+jXFDVNe61BeY=,2023-05-22 10:02:23+00:00,tpb_01,0,5821235,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,608924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,YRFewygp,AUWP7anI1ItYsAjFqTIMvwVvzjZRJZASn5osrpU=,2023-05-17 10:23:28+00:00,tpb_01,0,5793029,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,605654
5871,QiK9hJGz,ASknI5a2bE9f1CFObtZTYfgAJGJQ2ZIIiDdcPJk=,2023-05-16 09:27:59+00:00,tpb_01,0,5785883,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,604505
5872,aRFewWgx,AflQ70awBJ1d51V/oQNu+Uz0mFHeGcdYWwVGyyg=,2023-05-15 08:32:08+00:00,tpb_01,0,5778694,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,603452
5873,UiK9euT2,AeQL6Xte3uL7VpoFJdDcU41IMALiKJCF5FI5B8s=,2023-05-09 19:05:19+00:00,tpb_01,0,5746255,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,600611


In [232]:
vnpt_sent_list = vnpt_sent_list.merge(rq_vnpt[['telco_lead_id', 'vnpt_lead_id']], left_on = 'telco_lead_id', right_on='vnpt_lead_id', how='left')
vnpt_sent_list

,lead_unique_token,lead_phone_number,lead_created_at,product_code,lead_id,application_id,lead_telco_code,lead_source,mkt_channel,utm_campaign,smv_card_category,sms_offer_check,channel,uw_approved_date,card_issued_time,card_active_date,is_issued,is_actived,card_status,telco_lead_id_x,telco_lead_id_y,vnpt_lead_id
0,YRFezQ1M,AbtUALhVZg7R1i/HqMXINu6Ccthc2Jykngz6mNg=,2023-05-31 09:27:10+00:00,tpb_01,0,5874089,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,614534,1426465.0,614534.0
1,eRFeyEAU,AXW2hthsShnIqLkaH/+eecdDQbCELGuPtbMCPJc=,2023-05-24 11:43:10+00:00,tpb_01,0,5834754,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610322,1415574.0,610322.0
2,YRFeyBPM,AQforAYqU8wyPpIctS509TOTz+9Ft9V7ql3ciH8=,2023-05-24 08:56:50+00:00,tpb_01,0,5833337,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610091,1415075.0,610091.0
3,eRFex1-w,AXl7HrMYMu8ZevRazrDPrYGdhUm8Qpin1JjlqBQ=,2023-05-23 09:57:55+00:00,tpb_01,0,5827574,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,609569,1413536.0,609569.0
4,QiK9jTM6,ARNvC20nyCzrIvARB1JRGbF/SQ+jXFDVNe61BeY=,2023-05-22 10:02:23+00:00,tpb_01,0,5821235,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,608924,1411621.0,608924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,YRFewygp,AUWP7anI1ItYsAjFqTIMvwVvzjZRJZASn5osrpU=,2023-05-17 10:23:28+00:00,tpb_01,0,5793029,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,605654,1403485.0,605654.0
5871,QiK9hJGz,ASknI5a2bE9f1CFObtZTYfgAJGJQ2ZIIiDdcPJk=,2023-05-16 09:27:59+00:00,tpb_01,0,5785883,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,604505,1401015.0,604505.0
5872,aRFewWgx,AflQ70awBJ1d51V/oQNu+Uz0mFHeGcdYWwVGyyg=,2023-05-15 08:32:08+00:00,tpb_01,0,5778694,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,603452,1398673.0,603452.0
5873,UiK9euT2,AeQL6Xte3uL7VpoFJdDcU41IMALiKJCF5FI5B8s=,2023-05-09 19:05:19+00:00,tpb_01,0,5746255,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,600611,1389910.0,600611.0


In [155]:
vnpt_exclude_query = f""" 
SELECT DISTINCT 
  application_id
  , lead_unique_token
  , created_at
  , age + 1 as age
  , career
  , salary_payment_type
  , current_job
  , current_province
  , current_district
  , verdict_group 
FROM `prj-ts-p-analytic-8057.shared_tpb.tpb_dop_mapping`
WHERE lead_telco_code = 'vinaphone'
AND format_date('%Y-%m', date(created_at)) >= '2022-09'
AND (
  age < 21
  OR age > 64 
  OR current_job in ('Ban Hang Da Cap', 'Nhan Vien Quan Bar, Hop Dem, Karaoke, Massage', 'Nhan Vien Song Bac, Tiem Cam Do', 'Nhan Vien Thu Hoi No', 
  'Thuy Thu, Thuyen Vien, Nhan Vien Gian Khoan/Xa Bo', 'Huu Tri')
  OR (current_job in ('Phu Bep','Tai Xe','Bao Ve','Nhan Vien Giao Hang ','Nhan Vien Thu Cuoc','Nhan Vien Tiep Thi','Lao Cong Tap Vu','Boi Ban/Phuc Vu','Cong Nhan') AND salary_payment_type = 'Tien Mat')
  OR career = 'Khong Di Lam'
  OR (current_province not in ('Tinh An Giang', 'Tinh Ba Ria - Vung Tau', 'Tinh Bac Ninh', 'Tinh Binh Dinh', 'Tinh Binh Duong', 'Thanh Pho Can Tho', 'Thanh Pho Da Nang',
    'Tinh Dak Lak', 'Tinh Dong Nai', 'Thanh Pho Ha Noi', 'Thanh Pho Hai Phong', 'Thanh Pho Ho Chi Minh', 'Tinh Khanh Hoa', 'Tinh Kien Giang',
    'Tinh Lam Dong', 'Tinh Lao Cai', 'Tinh Nghe An', 'Tinh Quang Ninh', 'Tinh Tay Ninh', 'Tinh Thanh Hoa', 'Tinh Tien Giang','Not collected yet')
  and current_district not in ('Thanh Pho Thai Nguyen', 'Thi Xa Pho Yen', 'Huyen Dai Tu', 'Huyen Dong Hy', 'Thanh Pho Vinh Yen', 'Huyen Vinh Tuong', 
  'Huyen Binh Xuyen', 'Thanh Pho Bac Giang', 'Huyen Hiep Hoa', 'Huyen Viet Yen', 'Huyen Lang Giang', 'Huyen Luc Nam', 'Huyen Tan Yen',
  'Thanh Pho Hung Yen', 'Huyen Yen My', 'Huyen Khoai Chau', 'Huyen Van Giang', 'Huyen Quynh Phu','Huyen Hung Ha','Huyen Kien Xuong',
  'Thanh Pho Thai Binh','Huyen Thai Thuy','Huyen Dong Hung', 'Thanh Pho Hue', 'Thanh Pho Tam Ky', 'Thi Xa Dien Ban', 'Thi Xa Dong Xoai',
  'Huyen Chon Thanh', 'Thi Xa Phuoc Long', 'Thi Xa Binh Long', 'Huyen Dong Phu', 'Huyen Duc Hoa'))
  OR verdict_group like '%cic%'
)
ORDER BY 1,2;
"""

vnpt_exclude_list = bq_repo.get_data_from_query_into_pandas(query_str= vnpt_exclude_query)

In [233]:
vnpt_sent_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5875 entries, 0 to 5874
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   lead_unique_token  5875 non-null   object             
 1   lead_phone_number  5875 non-null   object             
 2   lead_created_at    5875 non-null   datetime64[ns, UTC]
 3   product_code       5875 non-null   object             
 4   lead_id            5875 non-null   Int64              
 5   application_id     5875 non-null   Int64              
 6   lead_telco_code    5875 non-null   object             
 7   lead_source        5875 non-null   object             
 8   mkt_channel        5875 non-null   object             
 9   utm_campaign       51 non-null     object             
 10  smv_card_category  337 non-null    object             
 11  sms_offer_check    5875 non-null   object             
 12  channel            5875 non-null   object       

In [234]:
transform_tpb_sms_map(vnpt_sent_list)

,lead_unique_token,lead_phone_number,lead_created_at,product_code,lead_id,application_id,lead_telco_code,lead_source,mkt_channel,utm_campaign,smv_card_category,sms_offer_check,channel,uw_approved_date,card_issued_time,card_active_date,is_issued,is_actived,card_status,telco_lead_id_x,telco_lead_id_y,vnpt_lead_id,cycle_active,lead_created_date,card_issued_date,fee_no_vat,fee_w_vat
0,YRFezQ1M,AbtUALhVZg7R1i/HqMXINu6Ccthc2Jykngz6mNg=,2023-05-31 09:27:10+00:00,tpb_01,0,5874089,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,614534,1426465.0,614534.0,NaN,2023-05-31,NaT,0,0.0
1,eRFeyEAU,AXW2hthsShnIqLkaH/+eecdDQbCELGuPtbMCPJc=,2023-05-24 11:43:10+00:00,tpb_01,0,5834754,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610322,1415574.0,610322.0,NaN,2023-05-24,NaT,0,0.0
2,YRFeyBPM,AQforAYqU8wyPpIctS509TOTz+9Ft9V7ql3ciH8=,2023-05-24 08:56:50+00:00,tpb_01,0,5833337,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610091,1415075.0,610091.0,NaN,2023-05-24,NaT,0,0.0
3,eRFex1-w,AXl7HrMYMu8ZevRazrDPrYGdhUm8Qpin1JjlqBQ=,2023-05-23 09:57:55+00:00,tpb_01,0,5827574,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,609569,1413536.0,609569.0,NaN,2023-05-23,NaT,0,0.0
4,QiK9jTM6,ARNvC20nyCzrIvARB1JRGbF/SQ+jXFDVNe61BeY=,2023-05-22 10:02:23+00:00,tpb_01,0,5821235,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,608924,1411621.0,608924.0,NaN,2023-05-22,NaT,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,YRFewygp,AUWP7anI1ItYsAjFqTIMvwVvzjZRJZASn5osrpU=,2023-05-17 10:23:28+00:00,tpb_01,0,5793029,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,605654,1403485.0,605654.0,NaN,2023-05-17,NaT,0,0.0
5871,QiK9hJGz,ASknI5a2bE9f1CFObtZTYfgAJGJQ2ZIIiDdcPJk=,2023-05-16 09:27:59+00:00,tpb_01,0,5785883,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,604505,1401015.0,604505.0,NaN,2023-05-16,NaT,0,0.0
5872,aRFewWgx,AflQ70awBJ1d51V/oQNu+Uz0mFHeGcdYWwVGyyg=,2023-05-15 08:32:08+00:00,tpb_01,0,5778694,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,603452,1398673.0,603452.0,NaN,2023-05-15,NaT,0,0.0
5873,UiK9euT2,AeQL6Xte3uL7VpoFJdDcU41IMALiKJCF5FI5B8s=,2023-05-09 19:05:19+00:00,tpb_01,0,5746255,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,600611,1389910.0,600611.0,NaN,2023-05-09,NaT,0,0.0


In [235]:
vnpt_monthly_minfee_qualified = pd.merge(vnpt_sent_list, vnpt_exclude_list.drop(columns=['created_at', 'application_id']), on='lead_unique_token', how='left', indicator=True)
list_excl_qualified = vnpt_monthly_minfee_qualified[(vnpt_monthly_minfee_qualified['_merge'] == 'both') &(vnpt_monthly_minfee_qualified['is_actived'] == 1)].drop(columns='_merge')

In [236]:
vnpt_monthly_minfee_qualified = vnpt_monthly_minfee_qualified.loc[vnpt_monthly_minfee_qualified['_merge'] == 'left_only', :].drop(columns='_merge')

In [237]:
vnpt_monthly_minfee_qualified['lead_created_month'] = vnpt_monthly_minfee_qualified['lead_created_date'].apply(lambda x: x.strftime('%Y-%m'))
vnpt_monthly_minfee_qualified['card_activated_month'] = vnpt_monthly_minfee_qualified['card_active_date'].apply(lambda x: x.strftime('%Y-%m') if not pd.isnull(x) else 'No-activate')
vnpt_monthly_minfee_qualified

,lead_unique_token,lead_phone_number,lead_created_at,product_code,lead_id,application_id,lead_telco_code,lead_source,mkt_channel,utm_campaign,smv_card_category,sms_offer_check,channel,uw_approved_date,card_issued_time,card_active_date,is_issued,is_actived,card_status,telco_lead_id_x,telco_lead_id_y,vnpt_lead_id,cycle_active,lead_created_date,card_issued_date,fee_no_vat,fee_w_vat,age,career,salary_payment_type,current_job,current_province,current_district,verdict_group,lead_created_month,card_activated_month
0,YRFezQ1M,AbtUALhVZg7R1i/HqMXINu6Ccthc2Jykngz6mNg=,2023-05-31 09:27:10+00:00,tpb_01,0,5874089,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,614534,1426465.0,614534.0,NaN,2023-05-31,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
1,eRFeyEAU,AXW2hthsShnIqLkaH/+eecdDQbCELGuPtbMCPJc=,2023-05-24 11:43:10+00:00,tpb_01,0,5834754,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,610322,1415574.0,610322.0,NaN,2023-05-24,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
3,eRFex1-w,AXl7HrMYMu8ZevRazrDPrYGdhUm8Qpin1JjlqBQ=,2023-05-23 09:57:55+00:00,tpb_01,0,5827574,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,609569,1413536.0,609569.0,NaN,2023-05-23,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
4,QiK9jTM6,ARNvC20nyCzrIvARB1JRGbF/SQ+jXFDVNe61BeY=,2023-05-22 10:02:23+00:00,tpb_01,0,5821235,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,608924,1411621.0,608924.0,NaN,2023-05-22,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
5,aRFezBtx,AZ7pvW3X2StirvvSLJ4rXCilfDWzO9yUo3aOdIw=,2023-05-29 23:10:09+00:00,tpb_01,0,5866350,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,613724,1424308.0,613724.0,NaN,2023-05-29,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5867,QiK9gNEK,AWbMXntqZ23YW6a06SX8Yh6w6GnRQN9o7tll3dE=,2023-05-13 14:45:37+00:00,tpb_01,0,5770512,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,602501,1396225.0,602501.0,NaN,2023-05-13,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
5868,YRFewEzX,AXzaJ/5PQEMCGDE3IQh77o/6mNzSqT58aYfWLXw=,2023-05-13 12:00:45+00:00,tpb_01,0,5769626,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,602399,1395970.0,602399.0,NaN,2023-05-13,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
5871,QiK9hJGz,ASknI5a2bE9f1CFObtZTYfgAJGJQ2ZIIiDdcPJk=,2023-05-16 09:27:59+00:00,tpb_01,0,5785883,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,604505,1401015.0,604505.0,NaN,2023-05-16,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate
5873,UiK9euT2,AeQL6Xte3uL7VpoFJdDcU41IMALiKJCF5FI5B8s=,2023-05-09 19:05:19+00:00,tpb_01,0,5746255,vinaphone,esms,SMS-VNPT WITH OFFER,None,None,GET_SMS_OFFER,SMS WITH OFFER,NaT,NaT,NaT,0,0,None,600611,1389910.0,600611.0,NaN,2023-05-09,NaT,0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,2023-05,No-activate


In [238]:
vnpt_minfee_summary = (vnpt_monthly_minfee_qualified
                       .groupby(['lead_created_month',])
                       .agg(qualified_leads=('lead_unique_token', 'nunique'),
                            cards_activated=('is_actived', 'sum'))
                       .reset_index()
                      )
vnpt_minfee_summary['activation_rate'] = vnpt_minfee_summary['cards_activated'] / vnpt_minfee_summary['qualified_leads']
vnpt_minfee_summary


,lead_created_month,qualified_leads,cards_activated,activation_rate
0,2023-05,4798,303,0.063151


In [239]:
vnpt_minfee_summary_break_month = (vnpt_monthly_minfee_qualified
                       .groupby(['lead_created_month', 'card_activated_month'])
                       .agg(qualified_leads=('lead_unique_token', 'nunique'),
                            cards_activated=('is_actived', 'sum'))
                       .reset_index()
                      )
vnpt_minfee_summary_break_month

,lead_created_month,card_activated_month,qualified_leads,cards_activated
0,2023-05,2023-05,222,222
1,2023-05,2023-06,78,78
2,2023-05,2023-07,2,2
3,2023-05,2023-08,1,1
4,2023-05,No-activate,4495,0


In [240]:
#write summary
ws.update('H20', 'VNPT min fee')
ggsheet_repo.write_df_to_sheet(ws, vnpt_minfee_summary, "H21")

ws.update('H30', 'VNPT min fee detail')
ggsheet_repo.write_df_to_sheet(ws, vnpt_minfee_summary_break_month, "H31")

In [241]:
vnpt_monthly_minfee_df =vnpt_monthly_minfee_qualified\
    .loc[:, ['lead_telco_code','lead_unique_token','telco_lead_id_x','lead_created_month','lead_created_date','card_issued_date','card_active_date','cycle_active', 'card_status']] \
    .rename(columns={'telco_lead_id_x': 'vnpt_lead_id'}).drop_duplicates()

In [242]:
#export to xlsx, drive and summary
vnpt_monthly_minfee_df.to_excel(f"{path}{recon_month}/TPB_VNPT_minfee_new_{minfee_month}.xlsx", index=False)
#export to drive

TPB_minfee_VNPT_wb = create_or_get_workbook(parent_folder_id = folder_id, wb_name = "TPB_VNPT_minfee_new_" + minfee_month)
create_or_update_sheet(wb = TPB_minfee_VNPT_wb, new_sheet = "Details", input_df = vnpt_monthly_minfee_df)

# create_or_get_folder_sheet(recon_month=recon_month, df=vnpt_monthly_minfee_df, sheet_name="TPB_VNPT_minfee_new_" + recon_month, parent_folder_id=parent_folder_id)
